In [ ]:
# === 1. Instalar dependências (1ª execução) ===
!pip install statsmodels openpyxl pytrends -q


In [ ]:
# === 2. Upload do arquivo Excel ===
from google.colab import files
uploaded = files.upload()
file_path = next(iter(uploaded))


In [ ]:
# === 3. Carregar e padronizar colunas ===
import pandas as pd
import unicodedata

def normalizar_colunas(df):
    df.columns = [
        unicodedata.normalize('NFKD', col).encode('ASCII', 'ignore').decode('utf-8')
        .strip().lower().replace(' ', '_')
        for col in df.columns
    ]
    return df

xls = pd.ExcelFile(file_path)
df = xls.parse('VENDA')
df = normalizar_colunas(df)
df_estoque = xls.parse('ESTOQUE')
df_estoque = normalizar_colunas(df_estoque)

# Corrigir coluna com erro de digitação se existir
if 'tamanho_produot' in df.columns:
    df = df.rename(columns={'tamanho_produot': 'tamanho_produto'})


In [ ]:
# === 4. Preparar datas ===
from dateutil.relativedelta import relativedelta
import pandas as pd

meses_ordem = {
    'janeiro': 1, 'fevereiro': 2, 'marco': 3, 'abril': 4,
    'maio': 5, 'junho': 6, 'julho': 7, 'agosto': 8,
    'setembro': 9, 'outubro': 10, 'novembro': 11, 'dezembro': 12
}

# Mapear meses e filtrar linhas inválidas
df['mes_num'] = df['mes_venda'].str.lower().map(meses_ordem)
df = df.dropna(subset=['ano_venda', 'mes_num'])
df['mes_num'] = df['mes_num'].astype(int)

df['ano_mes'] = pd.to_datetime(
    df['ano_venda'].astype(int).astype(str) + '-' + df['mes_num'].astype(str) + '-01',
    format='%Y-%m-%d'
)


In [ ]:
# === 5. Buscar tendências via Google Trends com termos genéricos ===
from pytrends.request import TrendReq
import time

def get_trend_uplift(linhas_otb):
    pytrends = TrendReq(hl='pt-BR', tz=360)
    genericos = [
        'acessorios', 'alpargata', 'anabela', 'mocassim', 'bolsa', 'bota', 'cinto', 'loafer', 'rasteira',
        'sandalia', 'sapatilha', 'scarpin', 'tenis', 'meia', 'meia pata', 'mest', 'salto', 'salto fino',
        'salto normal', 'sapato tratorado', 'mule', 'oxford', 'papete', 'peep flat', 'slide'
    ]

    tendencias = {}
    for linha in linhas_otb:
        try:
            # Tentar termos mais específicos (linha_otb) + genéricos
            termos_busca = [linha.lower()] + genericos
            pytrends.build_payload(termos_busca, cat=0, timeframe='today 3-m', geo='BR')
            df_trends = pytrends.interest_over_time()
            if not df_trends.empty:
                media_base = df_trends[linha.lower()] if linha.lower() in df_trends.columns else pd.Series([0])
                media_base = media_base.mean() if not media_base.empty else 0
                media_genericos = df_trends[genericos].mean(axis=1).mean()
                uplift = ((media_base + media_genericos) / 2 - 50) / 100
                tendencias[linha] = round(uplift, 3)
            else:
                # Se sem dados, usar média dos genéricos
                pytrends.build_payload(genericos, cat=0, timeframe='today 3-m', geo='BR')
                df_trends = pytrends.interest_over_time()
                media_genericos = df_trends[genericos].mean(axis=1).mean() if not df_trends.empty else 50
                uplift = (media_genericos - 50) / 100
                tendencias[linha] = round(uplift, 3)
            time.sleep(1)  # evitar bloqueios
        except Exception as e:
            # Em caso de erro, usar zero
            tendencias[linha] = 0
    return tendencias

linhas_otb_unicas = df['linha_otb'].dropna().unique().tolist()
trend_uplift = get_trend_uplift(linhas_otb_unicas)


In [ ]:
# === 6. Função de forecast ===
from statsmodels.tsa.api import ExponentialSmoothing
from dateutil.relativedelta import relativedelta
import pandas as pd

def forecast_serie(serie, passos=3):
    if serie.count() >= 6:
        modelo = ExponentialSmoothing(serie, trend='add', seasonal=None)
        modelo_fit = modelo.fit()
        previsao = modelo_fit.forecast(passos)
    else:
        previsao = pd.Series(
            [serie.mean()] * passos,
            index=pd.date_range(serie.index[-1] + relativedelta(months=1), periods=passos, freq='MS')
        )
    return previsao

last_date = df['ano_mes'].max()
periodos_forecast = 3
datas_previstas = pd.date_range(last_date + relativedelta(months=1), periods=periodos_forecast, freq='MS')


In [ ]:
# === 7. Previsão linha_otb × cor_produto com recomendação de estoque ===
resultado = []
for (linha_otb, cor_produto), grupo in df.groupby(['linha_otb', 'cor_produto']):
    serie = grupo.groupby('ano_mes')['qtd_vendida'].sum().reindex(df['ano_mes'].sort_values().unique(), fill_value=0)
    serie.index.freq = 'MS'
    prev = forecast_serie(serie, passos=periodos_forecast)
    
    g = trend_uplift.get(linha_otb, 0)
    prev_adj = prev * (1 + g)
    estoque_rec = (prev_adj.mean() * 2.8).round()

    estoque_atual = df_estoque[(df_estoque['linha'] == linha_otb) & (df_estoque['cor'] == cor_produto)]['saldo_empresa'].sum()

    registro = {
        'linha_otb': linha_otb,
        'cor_produto': cor_produto,
        'estoque_atual': estoque_atual
    }
    for dt_prev, val in prev_adj.items():
        col = f'venda_prevista_{dt_prev.strftime('%Y_%m')}'
        registro[col] = round(val, 0)
    registro['estoque_recomendado_total'] = int(round(estoque_rec, 0))
    resultado.append(registro)

# === 8. Exportar Excel final ===
df_resultado = pd.DataFrame(resultado)
meta_cols = ['linha_otb', 'cor_produto', 'estoque_atual']
for dt in datas_previstas:
    meta_cols.append(f'venda_prevista_{dt.strftime('%Y_%m')}')
meta_cols.append('estoque_recomendado_total')
df_resultado = df_resultado[meta_cols]

output_file = 'forecast_sugestao_compras.xlsx'
df_resultado.to_excel(output_file, index=False)

from google.colab import files
files.download(output_file)

print("\n✅ Arquivo gerado e pronto para download:", output_file)
display(df_resultado.head())
